In [1]:
from datasketch import MinHashLSHForest,MinHash,MinHashLSH,MinHashLSHEnsemble
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [2]:
sentences = [line.replace('\u200b','').strip() for line in open('weibos.txt').readlines()]

In [3]:
stop = [line.strip() for line in open('stopword.txt').readlines()]

In [5]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = []
    item = (pseg.cut(item_text))
    for i in list(item):
        # 去掉停用词
        if i.word not in list(stop):
            item_str.append(i.word)
    return item_str

# 对item_str创建MinHash


def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp


# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/kb/mz5fb0651t3b4jjc3w48t8s80000gn/T/jieba.cache
Loading model cost 0.676 seconds.
Prefix dict has been built succesfully.


In [7]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()

In [9]:
query = '国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职。'
# 将item_text进行分词
item_str = get_item_str(query)
# print(item_str)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 3)
for i in result:
    print(i, minhash_query.jaccard(minhash_list[i]), sentences[i])
print("Top 3 邻居", result)

16 1.0 国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职。
1 0.3359375 国足输给叙利亚之后，里皮辞职。谁将成为新主帅，成为广大球迷关注的焦点。目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里。
13 0.5234375 国足输给叙利亚后，里皮坐不住了，直接辞职了
Top 3 邻居 [16, 1, 13]
